In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00


In [5]:
from flask import Flask, request, jsonify
import openai
import time
import pandas as pd

app = Flask(__name__)

# Suggested to set up your OpenAI API key, below one is from a free version with limited access
openai.api_key = "YOUR_OPENAI_API_KEY"

# Initial conversation with system message
conversation = [
    {"role": "system", "content": "You are a helpful assistant that gathers user details."},
]



In [6]:
# Function to interact with chat agents with rate limiting handling
def chat_with_agent(conversation, message):
    conversation.append({"role": "user", "content": message})
    retries = 3

    while retries > 0:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_reply = response.choices[0].message["content"]
            conversation.append({"role": "assistant", "content": assistant_reply})
            return assistant_reply
        except openai.error.RateLimitError as e:
            print("Rate limit exceeded. Retrying in 10 seconds...")
            time.sleep(10)  # Wait for 10 seconds before retrying
            retries -= 1

    print("Failed to get a response after retries.")
    return None

In [7]:
# Function to gather user details using designed conversation flow
def gather_user_details():
    print("Welcome to the User Details Collection System!")
    user_details = {}
    collected_details = set()

    for detail, prompt in agent_prompts.items():

        # Ask for the current detail only if it hasn't been provided yet
        if detail not in collected_details:
            assistant_reply = chat_with_agent(conversation, prompt)
            print("Agent:", assistant_reply)
            user_response = input("User: ")
            user_details[detail] = user_response
            collected_details.add(detail)

    print("Thank you for providing your information!")
    return user_details


In [8]:
# Chat agent roles and their prompts
agent_prompts = {
    "name": "Hello! To get started, could you kindly let me know your full name?",
    "email": "Thank you for sharing your name. May I have your email address?",
    "phone": "Excellent! To stay connected, could you provide your phone number?",
    "address": "Thanks for sharing your contact details. What is your current address?",
    "dob": "Got it! When were you born? Please share your date of birth.",
    "education": "Thank you for providing your date of birth. Lastly, what is your highest level of education?",
}

In [ ]:
# Function to gather user details using REST API
@app.route("/collect_user_details", methods=["POST"])
def collect_user_details():
    try:
        user_details = gather_user_details()
        user_details_df = pd.DataFrame(user_details.items(), columns=["Detail", "Response"])
        csv_filename = "user_details.csv"
        user_details_df.to_csv(csv_filename, index=False)
        return jsonify({"message": "User details collected and saved to CSV."}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
